In [1]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

#0.1 Single Agent

In [2]:
import numpy as np
from scipy.special import logsumexp
from scipy.special import softmax
theta1 = np.array([[[4.57,3.41],[3.86,3.18]],[[2.69,2.75],[3.58,4.57]]]) #s,a1,a2

trans = 0.5*np.ones([2,2,2,2])      #s_t,a1,a2,s_{t+1}
trans[0,0,0,0] = trans[1,1,1,1] = 0.9
trans[0,0,0,1] = trans[1,1,1,0] = 0.1

Time= 100
SamplePath = 100

def ValueIteration(theta1 = theta1, trans= trans, beta = 0.9,gamma = 0.5772,thread_inner = 1e-7,thread_outer = 1e-7):
  pi1_new = 0.5*np.ones([2,2]) #s,a1
  Q1_new = np.zeros([2,2]) #s,a2
  Q2_new = np.zeros([2,2])

  #pi1_old = np.zeros([2,2])
  for ol in range(1000):
    pi1_old = pi1_new.copy()
    for il in range(1000):
      Q2_old = Q2_new.copy()
      Q2_tep = theta1 + beta * trans.dot(gamma + logsumexp(Q2_old,axis=1))
      Q2_new[:,0] = np.sum(pi1_old * Q2_tep[:,:,0],axis=1)
      Q2_new[:,1] = np.sum(pi1_old * Q2_tep[:,:,1],axis=1)
      if np.max(np.abs((Q2_new-Q2_old)/Q2_old)) <thread_inner: 
        #print(il,np.max(np.abs((Q2_new-Q2_old)/Q2_old)))
        break
    pi2_new = softmax(Q2_new,axis=1)
    pi2_old = pi2_new.copy()
    for il in range(1000):
      Q1_old = Q1_new.copy()
      Q1_tep = theta1 + beta * trans.dot(gamma + logsumexp(Q1_old,axis=1))
      Q1_new[:,0] = np.sum(pi2_old * Q1_tep[:,0,:],axis=1)
      Q1_new[:,1] = np.sum(pi2_old * Q1_tep[:,1,:],axis=1)
      if np.max(np.abs((Q1_new-Q1_old)/Q1_old)) <thread_inner:
        #print(il,np.max(np.abs((Q1_new-Q1_old)/Q1_old)))
        break     
    pi1_new = softmax(Q1_new,axis=1)

    if np.max(np.abs((pi1_old-pi1_new)/pi1_new)) <thread_outer:
      #print(ol,np.max(np.abs((pi1_old-pi1_new)/pi1_new)))
      break
  return Q1_new,Q2_new,pi1_new,pi2_new

def DataGeneration(theta1 = theta1,trans = trans,T=Time,N=SamplePath):
  state = np.zeros([N,T],dtype= int)
  action1 = np.zeros([N,T],dtype = int)
  action2 = np.zeros([N,T],dtype = int)

  state[:,0] = np.random.randint(0,2,size=N)
  _,_,pi1,pi2 = ValueIteration(theta1=theta1,trans=trans)
  for t in range(T):
    action_mat1 = np.random.random(size=N)
    action1[:,t] = np.int_((pi1[state[:,t],0])<=action_mat1)
    action_mat2 = np.random.random(size=N)
    action2[:,t] = np.int_((pi2[state[:,t],0])<=action_mat2)
    if t!=T-1:
      state_mat = np.random.random(size=N)
      state[:,t+1] = np.int_(trans[state[:,t],action1[:,t],action2[:,t],0]<=state_mat)
  return state, action1,action2 

state,action1,action2 = DataGeneration()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in true_divide


In [3]:
from scipy.optimize import minimize
def func1(trans,state = state, action1=action1,action2 = action2,T=Time):
  trans = np.vstack([trans,1-trans]).T.reshape([2,2,2,2]) #st,a1,a2,st+1
  return -np.sum(np.log(trans[state[:,0:-1],action1[:,0:-1],action2[:,0:-1],state[:,1:T]]))

res1 = minimize(func1,0.5*np.ones(8),#trans[:,:,:,0].reshape(-1), bounds=[[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1],[0,1]],
                method='Nelder-Mead')
trans_est = np.vstack([res1.x,1-res1.x]).T.reshape([2,2,2,2])
#print(res1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in log
  after removing the cwd from sys.path.


In [4]:
def func2(theta1,trans = trans,action1 = action1,action2 = action2,T=Time):
  theta1 = theta1.reshape([2,2,2])
  _,_,pi1,pi2 = ValueIteration(theta1=theta1,trans=trans)
  return - np.sum(np.log(pi1[state,action1])) -np.sum(np.log(pi2[state,action2]))

res2 = minimize(func2,theta1.reshape(-1),method='Nelder-Mead')
print(res2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in true_divide


 final_simplex: (array([[4.56193842, 3.35463217, 3.89935174, 3.18127308, 2.74235798,
        2.71504701, 3.68741264, 4.55711493],
       [4.5619465 , 3.35457285, 3.89930034, 3.18126904, 2.74230935,
        2.71512617, 3.68745275, 4.55711655],
       [4.56195255, 3.35460182, 3.89932543, 3.18129898, 2.74230269,
        2.7150676 , 3.68745448, 4.55711388],
       [4.56194304, 3.35461383, 3.89937207, 3.18129044, 2.74232892,
        2.71505628, 3.68737483, 4.55711475],
       [4.56193364, 3.35458727, 3.89934136, 3.18129434, 2.74231883,
        2.71506335, 3.68742488, 4.55712138],
       [4.56195145, 3.3545804 , 3.89938388, 3.18124403, 2.7422823 ,
        2.71508897, 3.68746048, 4.55711845],
       [4.56196024, 3.35455961, 3.89930902, 3.18127554, 2.74235952,
        2.71505937, 3.68742414, 4.55713204],
       [4.56192315, 3.35457267, 3.89932331, 3.18131629, 2.74236995,
        2.71503975, 3.68739989, 4.55708545],
       [4.56194832, 3.35454528, 3.89936008, 3.18128919, 2.74234847,
        2.7

In [5]:
def func2(theta1,trans = trans_est,action1 = action1,action2 = action2,T=Time):
  theta1 = theta1.reshape([2,2,2])
  _,_,pi1,pi2 = ValueIteration(theta1=theta1,trans=trans)
  return - np.sum(np.log(pi1[state,action1])) -np.sum(np.log(pi2[state,action2]))

res2 = minimize(func2,theta1.reshape(-1),method='Nelder-Mead')
print(res2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in true_divide
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:36: RuntimeWarning: divide by zero encountered in true_divide


 final_simplex: (array([[4.56150334, 3.34976894, 3.90302718, 3.19420652, 2.76167608,
        2.70000556, 3.68102336, 4.55976538],
       [4.56148965, 3.34980856, 3.90308938, 3.19420679, 2.76164474,
        2.69993509, 3.68102437, 4.55973705],
       [4.56147601, 3.3498241 , 3.90300412, 3.19420801, 2.76163373,
        2.70000798, 3.6809837 , 4.55972605],
       [4.56150052, 3.34979733, 3.90303267, 3.19427098, 2.76170366,
        2.69993339, 3.68103349, 4.55974219],
       [4.56145853, 3.34977326, 3.90306054, 3.19421656, 2.76167706,
        2.69999284, 3.68098988, 4.55972416],
       [4.56148621, 3.34982792, 3.90308831, 3.19421416, 2.76163453,
        2.70002377, 3.68099625, 4.55975779],
       [4.56147696, 3.34984698, 3.9029935 , 3.19418315, 2.76168076,
        2.70004646, 3.68106225, 4.55975475],
       [4.56149724, 3.34979305, 3.90308279, 3.1941093 , 2.76165841,
        2.70010235, 3.68103724, 4.55975014],
       [4.56145607, 3.34978831, 3.90301371, 3.19426784, 2.76159369,
        2.7